In [1]:
# Basic libraries
import numpy as np
import pandas as pd
# Preprocessing
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity


In [2]:
# Using base list of stop words - change?
count_vector = CountVectorizer(stop_words="english")

In [3]:
# Load datasets
steam_df = pd.read_csv('./steam_games.csv',delimiter=';', low_memory=False)

In [25]:
# Initial analysis
steam_df.tail()

,App ID,Name,Short Description,Developer,Publisher,Genre,Tags,Type,Categories,Owners,...,Price,Initial Price,Discount,CCU,Languages,Platforms,Release Date,Required Age,Website,Header Image
55680,999730,Secret Neighbor Beta,Beta Runs Aug 2-19! A group of kids tries to s...,"Hologryph, Dynamic Pixels",tinyBuild,"Action, Adventure, Indie","Multiplayer: 57, Horror: 54, Free to Play: 52,...",game,"Multi-player, Online PvP, PvP","200,000 .. 500,000",...,0,0,0,0,English,windows,2019/08/2,0,https://www.secretneighbor.com/,https://cdn.akamai.steamstatic.com/steam/apps/...
55682,999760,Mobile Wars X,Mobile Wars X is a shoot them up in which you ...,NAPE GAMES,NAPE GAMES,"Action, Casual, Free to Play, Indie","Free to Play: 25, Action: 24, Casual: 21, Indi...",game,Single-player,"20,000 .. 50,000",...,0,0,0,0,English,windows,2018/12/28,0,http://www.napegames.com/mobilewarsx/,https://cdn.akamai.steamstatic.com/steam/apps/...
55683,999830,Becalm,Fancy a relaxing drift through magical waters?...,Colorfiction,Colorfiction,"Casual, Free to Play, Indie","Free to Play: 56, Casual: 43, Indie: 43, Relax...",game,"Single-player, Full controller support","50,000 .. 100,000",...,0,0,0,0,"English, French, Italian, German, Spanish - Sp...","windows, mac, linux",2019/01/17,0,https://www.colorfiction.co/,https://cdn.akamai.steamstatic.com/steam/apps/...
55685,999860,Enemy On Board,Online (6v2) social deduction game that will r...,Windwalk Games,Windwalk Games,"Action, Casual, Free to Play, Indie, Early Access","Free to Play: 240, Indie: 233, Multiplayer: 22...",game,"Multi-player, Co-op, Online PvP, Online Co-op,...","100,000 .. 200,000",...,0,0,0,0,English,windows,2020/05/8,0,https://discord.gg/windwalk,https://cdn.akamai.steamstatic.com/steam/apps/...
55690,999990,Bouncing Hero,Bouncing Hero is an extreme auto-jump platform...,"ZPink, zniq.co",PsychoFlux Entertainment,"Action, Casual, Indie","Action: 42, Casual: 41, Indie: 41, Platformer:...",game,"Single-player, Steam Achievements, Steam Cloud","0 .. 20,000",...,499,499,0,0,"English, Japanese, Korean",windows,2019/09/1,0,http://www.bouncinghero.com/,https://cdn.akamai.steamstatic.com/steam/apps/...


In [ ]:
steam_df.dropna(inplace=True)
count_vector.fit(steam_df['Tags'])
doc_array = count_vector.transform(steam_df['Tags']).toarray()
frequency_df = pd.DataFrame(doc_array, columns = count_vector.get_feature_names_out())
similarity_df = pd.DataFrame(cosine_similarity(frequency_df))

In [19]:
steam_df.shape

(29548, 22)

In [21]:
frequency_df.shape

(29548, 3429)

In [20]:
similarity_df.shape

(29548, 29548)

In [26]:
similarity_df.tail(1)

,0,1,2,3,4,5,6,7,8,9,...,29538,29539,29540,29541,29542,29543,29544,29545,29546,29547
29547,0.070697,0.552158,0.059028,0.044672,0.032564,0.164053,0.058565,0.014377,0.296925,0.112757,...,0.046391,0.393174,0.049386,0.450835,0.120491,0.138039,0.33691,0.424183,0.058508,1.0


In [73]:
def get_similar_games(df, game_name, sim_df, thresh):
    game_info = sim_df.loc[df['Name'] == game_name, :]
    inds = game_info.loc[:,((game_info > thresh) & (game_info < 0.99999)).all()] #0.99999 is used because some games doing < 1 wasn't working and it would still find itself.
    t = df.iloc[inds.columns, :]
    i = inds.melt().set_index('variable')
    t['cosine_sim'] = i
    return t.dropna()

In [27]:
steam_df.reset_index(inplace=True)

In [34]:
get_similar_games(steam_df, 'Becalm', similarity_df, 0.5).sort_values('cosine_sim', ascending=False).head()

C:\Users\wally\AppData\Local\Temp\ipykernel_33440\3237803899.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  t['cosine_sim'] = i


,index,App ID,Name,Short Description,Developer,Publisher,Genre,Tags,Type,Categories,...,Initial Price,Discount,CCU,Languages,Platforms,Release Date,Required Age,Website,Header Image,cosine_sim
8763,19557,1653650,Lucid Dream Simulator,A very trippy escape room like game filled wit...,R3Games,R3Games,"Adventure, Casual, Indie, Strategy","Adventure: 69, Puzzle: 43, Exploration: 40, Co...",game,Single-player,...,499,0,0,English,windows,2021/06/18,0,https://discord.gg/p7ekRPxVP8,https://cdn.akamai.steamstatic.com/steam/apps/...,0.613320
14074,31132,331510,Amphora,Amphora is a peculiar puzzle game that mixes s...,Moondrop,Moondrop,"Adventure, Indie","Indie: 35, Puzzle: 33, Adventure: 30, Atmosphe...",game,"Single-player, Steam Achievements, Steam Cloud...",...,1399,0,0,"English, French, Italian, German, Spanish - Sp...","windows, mac",2014/11/12,0,http://amphoragame.com,https://cdn.akamai.steamstatic.com/steam/apps/...,0.556198
12591,29291,270230,Prominence,When the interstellar mission to colonize a ne...,Digital Media Workshop,Digital Media Workshop,"Adventure, Casual, Indie","Adventure: 47, Indie: 44, Casual: 43, Sci-fi: ...",game,"Single-player, Captions available, Steam Achie...",...,1999,0,0,"English, German",windows,2015/11/6,0,http://www.prominencegame.com,https://cdn.akamai.steamstatic.com/steam/apps/...,0.552368
5893,12302,1382180,Escape Knox Manor,Escape rooms are fun! But Escape Knox Manor as...,JTA Games,JTA Games,"Indie, Simulation","Experimental: 48, Retro: 42, Puzzle: 39, Atmos...",game,"Single-player, Steam Achievements",...,699,0,0,English,windows,2020/10/16,0,https://www.jtagames.net,https://cdn.akamai.steamstatic.com/steam/apps/...,0.550689
22526,42905,656020,Beta Runner,"Prepare yourself for the ultimate, &quot;run t...",Lightsoutgames,Lightsoutgames,"Adventure, Casual, Indie, Simulation","Casual: 21, Indie: 21, Adventure: 20, Simulati...",game,"Single-player, Stats, Steam Achievements",...,0,0,0,English,windows,2017/07/25,0,http://www.lightsoutgames.com,https://cdn.akamai.steamstatic.com/steam/apps/...,0.530994


In [74]:
get_similar_games(steam_df, 'Counter-Strike: Global Offensive', similarity_df, 0.3).sort_values('cosine_sim', ascending=False).head()

C:\Users\wally\AppData\Local\Temp\ipykernel_33440\2365979620.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  t['cosine_sim'] = i


,index,App ID,Name,Short Description,Developer,Publisher,Genre,Tags,Type,Categories,...,Initial Price,Discount,CCU,Languages,Platforms,Release Date,Required Age,Website,Header Image,cosine_sim
11667,28182,222880,Insurgency,Take to the streets for intense close quarters...,New World Interactive,New World Interactive,"Action, Indie, Strategy","FPS: 6589, Realistic: 2222, Tactical: 1963, Mu...",game,"Multi-player, Valve Anti-Cheat enabled, Co-op,...",...,1499,0,662,"English, French, German, Dutch, Polish, Portug...","windows, mac, linux",2014/01/22,0,https://www.newworldinteractive.com,https://cdn.akamai.steamstatic.com/steam/apps/...,0.426993
1274,2639,107410,Arma 3,Experience true combat gameplay in a massive m...,Bohemia Interactive,Bohemia Interactive,"Action, Simulation, Strategy","Action: 5572, Military: 5496, Multiplayer: 541...",game,"Multi-player, Single-player, Co-op, Captions a...",...,2999,0,17936,"English, French, Italian, German, Spanish - Sp...",windows,2013/09/12,0,http://www.arma3.com,https://cdn.akamai.steamstatic.com/steam/apps/...,0.421637
23109,43840,677620,Splitgate: Arena Warfare,"Splitgate is a free-to-play, fast-paced multip...",1047 Games,1047 Games,"Action, Free to Play, Indie","Free to Play: 607, FPS: 457, Multiplayer: 375,...",game,"Multi-player, Single-player, Captions availabl...",...,0,0,766,"English, French, Italian, German, Japanese, Po...","windows, linux",2019/05/24,0,https://1047games.com,https://cdn.akamai.steamstatic.com/steam/apps/...,0.413449
7,17,1000500,The Undisputables : Online Multiplayer Shooter,The Undisputables : Online Multiplayer is a Pv...,Amonga99,Amonga99,"Action, Casual, Indie, Massively Multiplayer","Multiplayer: 312, Shooter: 306, Third-Person S...",game,"Multi-player, Single-player, Co-op, Steam Achi...",...,699,0,0,"English, French, Italian, German, Spanish - Sp...","windows, mac, linux",2021/12/5,0,https://amonga99.com,https://cdn.akamai.steamstatic.com/steam/apps/...,0.407556
20797,40344,578080,PUBG: BATTLEGROUNDS,Play PUBG: BATTLEGROUNDS for free. Land on str...,"KRAFTON, Inc.","KRAFTON, Inc.","Action, Adventure, Free to Play, Massively Mul...","Survival: 14152, Shooter: 11956, Multiplayer: ...",game,"Multi-player, Stats, Online PvP, Remote Play o...",...,0,0,339287,"English, Korean, Simplified Chinese, French, G...",windows,2017/12/21,0,http://www.pubg.com,https://cdn.akamai.steamstatic.com/steam/apps/...,0.402015


In [65]:
get_similar_games(steam_df, 'RimWorld', similarity_df, 0.3).sort_values('cosine_sim', ascending=False).head()

C:\Users\wally\AppData\Local\Temp\ipykernel_33440\3237803899.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  t['cosine_sim'] = i


,index,App ID,Name,Short Description,Developer,Publisher,Genre,Tags,Type,Categories,...,Initial Price,Discount,CCU,Languages,Platforms,Release Date,Required Age,Website,Header Image,cosine_sim
17752,36011,455980,Judgment: Apocalypse Survival Simulation,"Build, scavenge, craft, and perform occult rit...",Suncrash,Suncrash,"Indie, Simulation, Strategy","Colony Sim: 250, Survival: 249, Base-Building:...",game,"Single-player, Steam Achievements, Steam Cloud...",...,1999,0,31,"English, German, Simplified Chinese, French, I...",windows,2018/05/3,0,http://www.judgmentgame.com,https://cdn.akamai.steamstatic.com/steam/apps/...,0.466569
23651,44756,700820,TFM: The First Men,A strategy simulation based on stories of the ...,Gathering Tree,Gathering Tree,"Indie, Simulation, Strategy, Early Access","Early Access: 360, Colony Sim: 327, Base-Build...",game,"Single-player, Steam Cloud",...,2499,0,117,English,windows,2022/11/2,0,http://playthefirstmen.com/,https://cdn.akamai.steamstatic.com/steam/apps/...,0.447129
29191,55004,979110,Space Haven,Embark on a space voyage with your ragtag crew...,Bugbyte Ltd.,Bugbyte Ltd.,"Indie, RPG, Simulation, Strategy, Early Access","Base-Building: 227, Space: 207, Colony Sim: 19...",game,"Single-player, Steam Cloud, Full controller su...",...,2299,0,369,"English, German, French, Spanish - Spain, Ital...","windows, mac, linux",2020/05/21,0,http://bugbyte.fi/spacehaven/,https://cdn.akamai.steamstatic.com/steam/apps/...,0.446362
16353,34119,403190,Planetbase,Guide a group of space settlers trying to esta...,Madruga Works,Madruga Works,"Indie, Simulation, Strategy","Colony Sim: 469, City Builder: 461, Base-Build...",game,"Single-player, Steam Achievements, Steam Tradi...",...,1499,0,210,"English, Spanish - Spain, French, German","windows, mac",2015/10/15,0,http://planetbase.madrugaworks.com,https://cdn.akamai.steamstatic.com/steam/apps/...,0.446362
18371,36820,484900,Aven Colony,Build a new home for humanity in Aven Colony. ...,Mothership Entertainment LLC,Team17 Digital Ltd,"Indie, Simulation, Strategy","Colony Sim: 94, City Builder: 88, Strategy: 84...",game,"Single-player, Steam Achievements, Steam Cloud...",...,2999,0,43,"English, French, German, Spanish - Spain, Russ...",windows,2017/07/25,0,http://AvenColony.com,https://cdn.akamai.steamstatic.com/steam/apps/...,0.440996


In [64]:
#Tool to find games if don't know exact name.
steam_df[steam_df['Name'].str.contains('Rim')]

,index,App ID,Name,Short Description,Developer,Publisher,Genre,Tags,Type,Categories,...,Price,Initial Price,Discount,CCU,Languages,Platforms,Release Date,Required Age,Website,Header Image
13101,29925,294100,RimWorld,A sci-fi colony sim driven by an intelligent A...,Ludeon Studios,Ludeon Studios,"Indie, Simulation, Strategy","Colony Sim: 3375, Base-Building: 3296, Surviva...",game,"Single-player, Steam Cloud, Steam Workshop, Re...",...,3499,3499,0,52984,"English, French, German, Polish, Russian, Ital...","windows, mac, linux",2018/10/17,0,https://ludeon.com,https://cdn.akamai.steamstatic.com/steam/apps/...
13531,30461,312600,Rime Berta,Command an army of puppets in this Japanese SR...,NEXT-SOFT+,PLAYISM,"RPG, Strategy","Turn-Based Tactics: 204, Tactical RPG: 197, Du...",game,"Single-player, Steam Achievements, Steam Tradi...",...,1499,1499,0,1,"English, Japanese",windows,2014/12/16,0,http://plus.next-soft.net/RB/,https://cdn.akamai.steamstatic.com/steam/apps/...
21428,41296,604640,Outer Rim,Alert...Alert… This is a distress message bein...,Outer Rim Studios LLC,Outer Rim Studios LLC,"Action, Casual, Indie","Casual: 22, Indie: 22, Action: 21",game,"Single-player, Partial Controller Support",...,99,99,0,0,English,windows,2017/03/30,0,http://www.defendtheouterrim.com,https://cdn.akamai.steamstatic.com/steam/apps/...
26230,49226,817360,The Outer Rim: Survivor,"Welcome to The Outer Rim, where fast paced act...",Katoa Games,Katoa Games,"Action, Free to Play, Indie, Early Access","Free to Play: 28, Action: 23, Indie: 22, Early...",game,"Single-player, Partial Controller Support",...,0,0,0,0,English,windows,2018/04/3,0,https://www.facebook.com/outerrimsurvivor/,https://cdn.akamai.steamstatic.com/steam/apps/...
